# Differential Privacy

Differential privacy is a method used to ensure individuals' privacy when sharing or analyzing data. The core idea is to add controlled random noise to the data or analysis results so that it becomes difficult to identify specific individuals, even if someone has access to the output of a data analysis. The amount of noise added is carefully controlled to balance privacy and accuracy, with a parameter called epsilon (ε) determining the level of privacy protection—smaller values of ε lead to stronger privacy but less accuracy.

This approach is used in various sectors like government surveys, tech companies, and medical research to protect sensitive information while still allowing valuable insights to be derived. By ensuring that the inclusion of any individual’s data doesn’t significantly impact the results, differential privacy helps organizations conduct data analysis without compromising personal privacy.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.flatten = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 5 * 5, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"\nusing device: {device}")

In [ ]:
num_epochs = 5
learning_rate = 0.001

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    total = 0
    correct = 0

    for images, labels in tqdm(
		iterable=train_loader,
		desc=f"[{epoch:{len(str(num_epochs))}d}/{num_epochs:{len(str(num_epochs))}d}]"
	):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    epoch_loss = running_loss / len(train_loader)
    train_losses.append(epoch_loss)

    print(f"--> loss={epoch_loss:.4f}, accuracy={accuracy:.2f}%\n")

print("Training complete!\n")

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(
        iterable=test_loader,
        desc="evaluate"
    ):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nTest Accuracy: {accuracy:.2f}%")

## Inversion Attack

In [ ]:
def inversion_attack(model, target_class, learning_rate, iterations, reg_param, l2_param):
    model.eval()

    recovered_image = torch.rand(1, 1, 28, 28, requires_grad=True, device=device)
    optimizer = optim.Adam([recovered_image], lr=learning_rate)

    target = torch.zeros(1, 10, device=device)
    target[0, target_class] = 1

    losses = []

    for i in tqdm(range(iterations), desc="inversion_attack"):
        optimizer.zero_grad()

        pred = model(recovered_image)
        classification_loss = F.cross_entropy(pred, target.argmax(dim=1))

        tv_loss = torch.sum(torch.abs(recovered_image[:, :, :, :-1] - recovered_image[:, :, :, 1:])) + \
                  torch.sum(torch.abs(recovered_image[:, :, :-1, :] - recovered_image[:, :, 1:, :]))

        tv_loss *= reg_param
        l2_loss = torch.sum(recovered_image ** 2) * l2_param
        total_loss = classification_loss + tv_loss + l2_loss

        total_loss.backward()
        optimizer.step()

        with torch.no_grad():
            recovered_image.clamp_(0, 1)

        if i % 500 == 499:
            losses.append(classification_loss.item())
            print(f'[{i + 1:4d}] loss: {classification_loss.item():.6f}')

    with torch.no_grad():
        image = recovered_image.cpu().squeeze().numpy()

    return (image, losses)

## Apply Noise Layer to Model

In [ ]:
noise_level = 0.13

noised_model = CNN().to(device)
noised_model.load_state_dict(model.state_dict())

with torch.no_grad():
    for param in noised_model.parameters():
        param.add_(torch.randn_like(param) * noise_level)

In [ ]:
noised_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(
        iterable=test_loader,
        desc="evaluate"
    ):
        images, labels = images.to(device), labels.to(device)

        outputs = noised_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"\nTest Accuracy: {accuracy:.2f}%")

## Attack Model and Noised Model

In [ ]:
target_class = 8

(image, losses) = inversion_attack(
	model=model,
	target_class=target_class,
	learning_rate=0.01,
	iterations=2000,
	reg_param=1e-5,
	l2_param=1e-3
)

(noised_image, noised_losses) = inversion_attack(
	model=noised_model,
	target_class=target_class,
	learning_rate=0.01,
	iterations=2000,
	reg_param=1e-5,
	l2_param=1e-3
)

## Plot Images

In [ ]:
plt.figure(figsize=(12, 4))
fig, axis = plt.subplots(1, 2)

axis[0].imshow(image, cmap='gray')
axis[0].axis('off')

axis[1].imshow(noised_image, cmap='gray')
axis[1].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axis = plt.subplots(2, 10, figsize=(13, 4))

for i in range(10):
    (image, losses) = inversion_attack(
        model=model,
        target_class=i,
        learning_rate=0.01,
        iterations=2000,
        reg_param=1e-5,
        l2_param=1e-3
    )

    (noised_image, noised_losses) = inversion_attack(
        model=noised_model,
        target_class=i,
        learning_rate=0.01,
        iterations=2000,
        reg_param=1e-5,
        l2_param=1e-3
    )

    axis[0, i].imshow(image, cmap='gray')
    axis[0, i].axis('off')

    axis[1, i].imshow(noised_image, cmap='gray')
    axis[1, i].axis('off')

plt.tight_layout()
plt.show()

# Federated Learning

In [ ]:
import numpy as np

class Client:
    def __init__(self, dataset, client_id, device):
        self.dataset = dataset
        self.client_id = client_id
        self.device = device
        self.model = CNN().to(device)
        self.dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    def train(self, epochs=1, noise_level=0.0):
        self.model.train()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        total_loss = 0

        total = 0
        correct = 0

        for epoch in range(epochs):
            total_loss = 0

            for data, target in tqdm(
                iterable=self.dataloader,
                desc=f"[{epoch:{len(str(epochs))}d}/{epochs:{len(str(epochs))}d}]"
            ):
                data, target = data.to(self.device), target.to(self.device)

                optimizer.zero_grad()
                output = self.model(data)
                loss = F.cross_entropy(output, target)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

        with torch.no_grad():
            for param in self.model.parameters():
                param.add_(torch.randn_like(param) * noise_level)

        self.model.eval()
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = 100 * correct / total
        return total_loss / len(self.dataloader), accuracy

    def evaluate(self, test_loader):
        self.model.eval()
        test_loss = 0
        correct = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        accuracy = 100. * correct / len(test_loader.dataset)

        return test_loss, accuracy

    def get_parameters(self):
        return {k: v.cpu() for k, v in self.model.state_dict().items()}

    def set_parameters(self, parameters):
        params_on_device = {k: v.to(self.device) for k, v in parameters.items()}
        self.model.load_state_dict(params_on_device)

In [ ]:
from collections import OrderedDict

class Server:
    def __init__(self, test_dataset, device):
        self.clients = []
        self.device = device
        self.global_model = CNN().to(device)
        self.test_loader = DataLoader(test_dataset, batch_size=128)

    def add_client(self, client):
        self.clients.append(client)

    def aggregate_parameters(self, client_parameters):
        global_dict = OrderedDict()

        for k in client_parameters[0].keys():
            global_dict[k] = torch.stack([client_parameters[i][k] for i in range(len(client_parameters))], 0).mean(0)

        return global_dict

    def update_global_model(self):
        client_parameters = [client.get_parameters() for client in self.clients]
        global_parameters = self.aggregate_parameters(client_parameters)

        self.global_model.load_state_dict({k: v.to(self.device) for k, v in global_parameters.items()})

        for client in self.clients:
            client.set_parameters(global_parameters)

        return client_parameters[0]  # Return client 1's parameters for potential attack

    def evaluate_global_model(self):
        self.global_model.eval()
        test_loss = 0
        correct = 0

        with torch.no_grad():
            for data, target in self.test_loader:
                data, target = data.to(self.device), target.to(self.device)
                output = self.global_model(data)
                test_loss += F.cross_entropy(output, target, reduction='sum').item()
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(self.test_loader.dataset)
        accuracy = 100. * correct / len(self.test_loader.dataset)

        return test_loss, accuracy

In [ ]:
def load_mnist():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST('./data', train=False, transform=transform)

    return train_dataset, test_dataset

In [ ]:
from torch.utils.data import Subset

def distribute_data(dataset, num_clients, iid=True):
    if iid:
        num_items_per_client = len(dataset) // num_clients
        client_datasets = []

        indices = torch.randperm(len(dataset))
        for i in range(num_clients):
            start_idx = i * num_items_per_client
            end_idx = (i + 1) * num_items_per_client if i < num_clients - 1 else len(dataset)
            client_indices = indices[start_idx:end_idx]
            client_datasets.append(Subset(dataset, client_indices))
    else:
        labels = dataset.targets.numpy()
        sorted_indices = np.argsort(labels)
        client_datasets = []
        shards_per_client = 2

        num_shards = num_clients * shards_per_client
        items_per_shard = len(dataset) // num_shards
        shard_indices = []

        for i in range(num_shards):
            start_idx = i * items_per_shard
            end_idx = (i + 1) * items_per_shard if i < num_shards - 1 else len(sorted_indices)
            shard_indices.append(sorted_indices[start_idx:end_idx])

        np.random.shuffle(shard_indices)

        for i in range(num_clients):
            client_idx = []
            for j in range(shards_per_client):
                client_idx.extend(shard_indices[i * shards_per_client + j])
            client_datasets.append(Subset(dataset, client_idx))

    return client_datasets

In [ ]:
def run_federated_learning(num_clients=3, num_rounds=3, local_epochs=1, noise_level=0.0, iid=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    train_dataset, test_dataset = load_mnist()
    client_datasets = distribute_data(train_dataset, num_clients, iid=iid)

    server = Server(test_dataset, device)
    clients = []
    client_params_list = []

    for i in range(num_clients):
        client = Client(client_datasets[i], i, device)
        clients.append(client)
        server.add_client(client)

    global_accuracies = []
    client_losses = [[] for _ in range(num_clients)]

    # Federated learning loop
    for round_num in range(num_rounds):
        client_params_list.append([])

        print(f"\nRound {round_num+1}/{num_rounds}")

        for i, client in enumerate(clients):
            loss, accuracy = client.train(epochs=local_epochs, noise_level=noise_level)
            client_losses[i].append(loss)
            print(f"[Client {i+1}] loss={loss:.4f}, accuracy={accuracy:.2f}%")

            client_params_list[round_num].append(client.get_parameters())

        server.update_global_model()
        test_loss, accuracy = server.evaluate_global_model()
        global_accuracies.append(accuracy)
        print(f"Global model - Test loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")

    return (global_accuracies, client_losses), (server, clients, client_params_list)

In [ ]:
num_clients = 10
num_rounds = 5

local_epochs = 1
noise_level = 0.0
iid = True

# Training

In [ ]:
_, (server, clients, _) = run_federated_learning(
    num_clients=num_clients,
    num_rounds=num_rounds,
    local_epochs=local_epochs,
    noise_level=noise_level,
    iid=iid
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_params = [clients[i].get_parameters() for i in range(num_clients)]

# Inversion Attack on Federated Learning

In [ ]:
import torch.nn.functional as F

class ModelInversionAttack:
    def __init__(self, model_params, target_class, device):
        self.model_params = model_params
        self.target_class = target_class
        self.device = device

    def inversion(self, target_name, num_iterations=1000, learning_rate=0.01, reg_param=0.01):
        model = CNN().to(self.device)
        model.load_state_dict({k: v.to(self.device) for k, v in self.model_params.items()})
        model.eval()

        recovered_image = torch.rand(1, 1, 28, 28, requires_grad=True, device=self.device)
        optimizer = optim.Adam([recovered_image], lr=learning_rate)

        target = torch.zeros(1, 10, device=self.device)
        target[0, self.target_class] = 1

        losses = []
        classification_loss = 0
        correct_predictions = 0

        for i in tqdm(range(num_iterations), desc=f"attack [{target_name}]"):
            optimizer.zero_grad()
            classification_loss = 0

            pred = model(recovered_image)
            pred_prob = F.softmax(pred, dim=1)
            classification_loss += -torch.sum(target * torch.log(pred_prob + 1e-10))

            predicted_class = torch.argmax(pred_prob, dim=1)
            if predicted_class == self.target_class:
                correct_predictions += 1

            tv_loss = torch.sum(torch.abs(recovered_image[:, :, :, :-1] - recovered_image[:, :, :, 1:])) + \
                      torch.sum(torch.abs(recovered_image[:, :, :-1, :] - recovered_image[:, :, 1:, :]))
            tv_loss = tv_loss * reg_param

            l2_loss = torch.sum(recovered_image ** 2) * 0.001

            total_loss = classification_loss + tv_loss + l2_loss

            total_loss.backward()
            optimizer.step()

            with torch.no_grad():
                recovered_image.clamp_(0, 1)

        accuracy = (correct_predictions / (num_iterations)) * 100

        print(f"--> loss={classification_loss.item():.4f}, accuracy={accuracy:.2f}%")

        with torch.no_grad():
            recovered_image_np = recovered_image.cpu().numpy()

        return recovered_image_np, losses

In [ ]:
results = []
target_class = 5 # Class to attack

for model_param in model_params:
    images = []

    attack = ModelInversionAttack(model_param, target_class, device)
    test_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, transform=transforms.ToTensor()), batch_size=64)

    attacked_image, _ = attack.inversion(
        target_name=target_class,
        num_iterations=400, # 1000
        learning_rate=0.1,  # 0.01
        reg_param=1e-4      # 1e-5
    )
    attacked_image = np.squeeze(attacked_image)

    images.append(attacked_image)
    results.append(images)

# Server attack
global_params =  {k: v.cpu() for k, v in server.global_model.state_dict().items()}

attack = ModelInversionAttack(global_params, target_class, device)
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./data', train=False, transform=transforms.ToTensor()), batch_size=64)

global_image, _ = attack.inversion(
    target_name=target_class,
    num_iterations=1000, # 1000
    learning_rate=0.01,  # 0.01
    reg_param=1e-5      # 1e-5
)
global_image = np.squeeze(global_image)

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(10, 6))
round = 0

for y in range(3):
    for x in range(4):
        if round <= 9:
            axes[y][x].imshow(results[round][0], cmap='gray')
            axes[y][x].set_title(f"Client {round + 1}")
        axes[y][x].axis("off")
        round += 1

axes[2][3].imshow(global_image, cmap='gray')
axes[2][3].set_title(f"Server")

plt.tight_layout()
plt.show()